<a href="https://colab.research.google.com/github/ssvibitha/SIH_Railways/blob/main/jsonToCsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()  # this opens a file picker


Saving schedules.json to schedules.json
Saving stations.json to stations.json
Saving timetable.csv to timetable.csv
Saving traffic_density.csv to traffic_density.csv
Saving trains.json to trains.json


In [ ]:
!git clone https://github.com/ssvibitha/SIH_Railways.git


Cloning into 'SIH_Railways'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 0), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 18.59 MiB | 12.93 MiB/s, done.


In [ ]:
import pandas as pd
import json

In [ ]:
# Load trains JSON
with open('trains.json', 'r') as f:
    trains_data = json.load(f)
trains = pd.DataFrame(trains_data)

# Load stations JSON
with open('stations.json', 'r') as f:
    stations_data = json.load(f)
stations = pd.DataFrame(stations_data)

# Load schedules JSON
with open('schedules.json', 'r') as f:
    schedules_data = json.load(f)
schedules = pd.DataFrame(schedules_data)

# Load timetable CSV
timetable = pd.read_csv('timetable.csv')

# Load traffic density CSV
traffic_density = pd.read_csv('traffic_density.csv')


/tmp/ipython-input-1738645134.py:17: DtypeWarning: Columns (0,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  timetable = pd.read_csv('timetable.csv')


In [ ]:
print("Train dataset")
print(trains.head())
print("\nStations dataset")
print(stations.head())
print("\nSchedules dataset")
print(schedules.head())
print("\nTimetable dataset")
print(timetable.head())
print("\nTraffic Density dataset")
print(traffic_density.head())

# Check for missing values
print(trains.isnull().sum())
print(stations.isnull().sum())
print(schedules.isnull().sum())


Train dataset
                type                                           features
0  FeatureCollection  {'geometry': {'type': 'LineString', 'coordinat...
1  FeatureCollection  {'geometry': {'type': 'LineString', 'coordinat...
2  FeatureCollection  {'geometry': {'type': 'LineString', 'coordinat...
3  FeatureCollection  {'geometry': {'type': 'LineString', 'coordinat...
4  FeatureCollection  {'geometry': {'type': 'LineString', 'coordinat...

Stations dataset
                type                                           features
0  FeatureCollection  {'geometry': {'type': 'Point', 'coordinates': ...
1  FeatureCollection  {'geometry': None, 'type': 'Feature', 'propert...
2  FeatureCollection  {'geometry': None, 'type': 'Feature', 'propert...
3  FeatureCollection  {'geometry': None, 'type': 'Feature', 'propert...
4  FeatureCollection  {'geometry': {'type': 'Point', 'coordinates': ...

Schedules dataset
  arrival  day                     train_name         station_name  \
0    None  1.0 

In [ ]:
# Example: fill missing arrival/departure with placeholder
timetable['Arrival time'].fillna('00:00')
timetable['Departure Time'].fillna('00:00')

,Departure Time
0,10:25:00
1,11:08:00
2,11:30:00
3,00:00:00
4,20:30:00
...,...
186119,23:31:00
186120,23:36:00
186121,23:40:00
186122,23:42:00


In [ ]:
# Standardize time formats
timetable['Arrival time'] = pd.to_datetime(timetable['Arrival time'], format='%H:%M', errors='coerce')
timetable['Departure Time'] = pd.to_datetime(timetable['Departure Time'], format='%H:%M', errors='coerce')

In [ ]:
# Map station codes consistently
station_map = dict(zip(stations['code'], stations['name']))
timetable['Station Name'] = timetable['Station Code'].map(station_map)
schedules['station_name'] = schedules['station_code'].map(station_map)

KeyError: 'code'

In [ ]:
import json
import pandas as pd

# Step 1: Load the JSON file
with open('stations.json', 'r') as f:
    data = json.load(f)

# Step 2: Flatten the 'features' list into a DataFrame
stations = pd.json_normalize(data['features'])

# Step 3: Extract relevant columns
# 'properties.code' → station code, 'properties.name' → station name, etc.
stations_clean = stations[['properties.code', 'properties.name', 'properties.state',
                           'properties.zone', 'properties.address', 'geometry.coordinates']]

# Step 4: Rename columns for clarity
stations_clean = stations_clean.rename(columns={
    'properties.code': 'station_code',
    'properties.name': 'station_name',
    'properties.state': 'state',
    'properties.zone': 'zone',
    'properties.address': 'address',
    'geometry.coordinates': 'coordinates'
})

# Step 5: Handle missing coordinates (optional)
stations_clean['coordinates'] = stations_clean['coordinates'].apply(lambda x: x if isinstance(x, list) else [None, None])

# Step 6: Export to CSV
stations_clean.to_csv('stations_clean.csv', index=False)

print("CSV saved successfully! Here are the first 5 rows:")
print(stations_clean.head())


CSV saved successfully! Here are the first 5 rows:
  station_code station_name          state  zone  \
0         BDHL       Badhal      Rajasthan   NWR   
1      XX-BECE      XX-BECE           None  None   
2      XX-BSPY      XX-BSPY           None  None   
3      YY-BPLC      YY-BPLC           None  None   
4          KHH       KICHHA  Uttar Pradesh   NER   

                        address                      coordinates  
0  Kishangarh Renwal, Rajasthan         [75.4516454, 27.2520587]  
1                          None                     [None, None]  
2                          None                     [None, None]  
3                          None                     [None, None]  
4         Kichha, Uttar Pradesh  [79.519746, 28.913427000000002]  


In [ ]:
from google.colab import files
files.download('stations_clean.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json
import pandas as pd

# Step 1: Load JSON file
with open('trains.json', 'r') as f:
    data = json.load(f)

# Step 2: Flatten 'features' list into a DataFrame
trains = pd.json_normalize(data['features'])

# Step 3: Extract relevant columns
# geometry.coordinates → route coordinates
# properties.* → train details
trains_clean = trains[['properties.number', 'properties.name', 'properties.type',
                       'properties.from_station_code', 'properties.from_station_name',
                       'properties.to_station_code', 'properties.to_station_name',
                       'properties.departure', 'properties.arrival', 'properties.duration_h',
                       'properties.duration_m', 'properties.distance', 'properties.zone',
                       'properties.first_class', 'properties.first_ac', 'properties.second_ac',
                       'properties.third_ac', 'properties.chair_car', 'properties.sleeper',
                       'properties.classes', 'properties.return_train',
                       'geometry.coordinates']]

# Step 4: Rename columns to snake_case
trains_clean = trains_clean.rename(columns={
    'properties.number': 'train_number',
    'properties.name': 'train_name',
    'properties.type': 'train_type',
    'properties.from_station_code': 'from_station_code',
    'properties.from_station_name': 'from_station_name',
    'properties.to_station_code': 'to_station_code',
    'properties.to_station_name': 'to_station_name',
    'properties.departure': 'departure_time',
    'properties.arrival': 'arrival_time',
    'properties.duration_h': 'duration_hours',
    'properties.duration_m': 'duration_minutes',
    'properties.distance': 'distance_km',
    'properties.zone': 'zone',
    'properties.first_class': 'first_class',
    'properties.first_ac': 'first_ac',
    'properties.second_ac': 'second_ac',
    'properties.third_ac': 'third_ac',
    'properties.chair_car': 'chair_car',
    'properties.sleeper': 'sleeper',
    'properties.classes': 'classes',
    'properties.return_train': 'return_train',
    'geometry.coordinates': 'route_coordinates'
})

# Step 5: Handle missing/empty values if needed
trains_clean['classes'] = trains_clean['classes'].fillna('')

# Step 6: Export to CSV
trains_clean.to_csv('trains_clean.csv', index=False)

print("Trains CSV saved successfully! Here are the first 5 rows:")
print(trains_clean.head())


Trains CSV saved successfully! Here are the first 5 rows:
  train_number                         train_name train_type  \
0        04601        Jammu Tawi Udhampur Special       DEMU   
1        04602             UDHAMPUR JAMMUTAWI DMU       DEMU   
2        04603                   JAT UDAHMPUR DMU       DEMU   
3        04604             UDHAMPUR JAMMUTAWI DMU       DEMU   
4        04728  Mumbai BandraT-Bikaner SF Special         SF   

  from_station_code       from_station_name to_station_code to_station_name  \
0               JAT              JAMMU TAWI             UHP        UDHAMPUR   
1               UHP                UDHAMPUR             JAT      JAMMU TAWI   
2               JAT              JAMMU TAWI             UHP        UDHAMPUR   
3               UHP                UDHAMPUR             JAT      JAMMU TAWI   
4              BDTS  MUMBAI BANDRA TERMINUS             BKN      BIKANER JN   

  departure_time arrival_time  duration_hours  ...  zone  first_class  \
0       1

In [ ]:
from google.colab import files
files.download('trains_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json
import pandas as pd

# Step 1: Load JSON file
with open('schedules.json', 'r') as f:
    data = json.load(f)

# Step 2: Convert list of dictionaries to DataFrame
schedules = pd.DataFrame(data)

# Step 3: Rename columns to snake_case (optional but consistent)
schedules = schedules.rename(columns={
    'train_name': 'train_name',
    'train_number': 'train_number',
    'station_name': 'station_name',
    'station_code': 'station_code',
    'arrival': 'arrival_time',
    'departure': 'departure_time',
    'day': 'day',
    'id': 'record_id'
})

# Step 4: Handle missing arrival times
schedules['arrival_time'] = schedules['arrival_time'].replace('None', pd.NA)

# Step 5: Export to CSV
schedules.to_csv('schedules_clean.csv', index=False)

print("Schedules CSV saved successfully! Here are the first 5 rows:")
print(schedules.head())


Schedules CSV saved successfully! Here are the first 5 rows:
  arrival_time  day                     train_name         station_name  \
0         <NA>  1.0     Falaknuma Lingampalli MMTS  KACHEGUDA FALAKNUMA   
1         <NA>  1.0   Thrissur Guruvayur Passenger              THRISUR   
2         <NA>  1.0  Porbandar Muzaffarpur Express            PORBANDAR   
3         <NA>  1.0             RAIPUR ITWARI PASS            RAIPUR JN   
4         <NA>  1.0             Gomoh-Asansol MEMU             GOMOH JN   

  station_code  record_id train_number departure_time  
0           FM     302214        47154       07:55:00  
1          TCR     281458        56044       18:55:00  
2          PBR     309335        19269       15:05:00  
3            R     283774        58205       13:30:00  
4          GMO     319937        63542       07:20:00  


In [ ]:
from google.colab import files
files.download('schedules_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Step 1: Load the CSV
timetable = pd.read_csv('timetable.csv')

# Step 2: Rename columns to snake_case
timetable = timetable.rename(columns={
    'Train No': 'train_no',
    'Train Name': 'train_name',
    'SEQ': 'seq',
    'Station Code': 'station_code',
    'Station Name': 'station_name',
    'Arrival time': 'arrival_time',
    'Departure Time': 'departure_time',
    'Distance': 'distance_km',
    'Source Station': 'source_station_code',
    'Source Station Name': 'source_station_name',
    'Destination Station': 'destination_station_code',
    'Destination Station Name': 'destination_station_name'
})

# Step 3: Save the updated CSV
timetable.to_csv('timetable_snake_case.csv', index=False)

print("Columns updated to snake_case! Here are the first 5 rows:")
print(timetable.head())


/tmp/ipython-input-1440165849.py:4: DtypeWarning: Columns (0,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  timetable = pd.read_csv('timetable.csv')


Columns updated to snake_case! Here are the first 5 rows:
  train_no    train_name seq station_code  station_name arrival_time  \
0      107  SWV-MAO-VLNK   1          SWV  SAWANTWADI R     00:00:00   
1      107  SWV-MAO-VLNK   2         THVM        THIVIM     11:06:00   
2      107  SWV-MAO-VLNK   3         KRMI       KARMALI     11:28:00   
3      107  SWV-MAO-VLNK   4          MAO   MADGOAN JN.     12:10:00   
4      108  VLNK-MAO-SWV   1          MAO   MADGOAN JN.     00:00:00   

  departure_time distance_km source_station_code source_station_name  \
0       10:25:00           0                 SWV     SAWANTWADI ROAD   
1       11:08:00          32                 SWV     SAWANTWADI ROAD   
2       11:30:00          49                 SWV     SAWANTWADI ROAD   
3       00:00:00          78                 SWV     SAWANTWADI ROAD   
4       20:30:00           0                 MAO         MADGOAN JN.   

  destination_station_code destination_station_name  
0                     

In [ ]:
from google.colab import files
files.download('timetable_snake_case.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>